In [86]:
import time
import random
import copy
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd

from torchtext import data

import util as util
from models import *
from deep_main import *

# %load_ext autoreload
# %autoreload 2


SEED = 1234
torch.manual_seed(SEED)

# CONFIGURE THESE PARAMETERS
# DEVELOPING = True
DEVELOPING = False
WHICH_TASK = "response" # also can be "product"

if DEVELOPING:
    # in order: train, validation, test
    files = ["../data/complaints_3k.csv", \
                "../data/complaints_500.csv", \
                "../data/complaints_1k.csv"]
    BATCH_SIZE = 7
    MAX_VOCAB_SIZE = 5000
else:
    # in order: train, validation, test
    files = ["../data/full_training_set.csv", \
                "../data/full_validation_set.csv", \
                "../data/full_testing_set.csv"]    
    BATCH_SIZE = 64
    MAX_VOCAB_SIZE = 25000

USE_CUDA = False
INPUT_DIM = MAX_VOCAB_SIZE + 2 # words + pad + unknown
NUM_EPOCHS = 1
GRAD_CLIP = 1

In [96]:
# TEXT = data.Field(sequential=True, tokenize=util.tokenize, lower=True)
# OneHotEncoder = data.Pipeline(convert_token=util.one_hot_encode_response)
# LABEL = data.LabelField(sequential=False, use_vocab=False, preprocessing=OneHotEncoder)
# train_data = load_and_tokenize_data(files[0], TEXT, LABEL, WHICH_TASK)
# TEXT.build_vocab(train_data, max_size=MAX_VOCAB_SIZE)

In [89]:
train_data, valid_data, test_data = preprocess(WHICH_TASK, *files)

train_iter, valid_iter, test_iter = data.BucketIterator.splits( \
(train_data, valid_data, test_data), \
sort_key = lambda x: len(x.narrative), \
sort_within_batch=False, \
batch_size = BATCH_SIZE) 

iters = (train_iter, valid_iter, test_iter)

'''
DO MODEL RUNS
'''


if WHICH_TASK == "response":
    parameters = {
        "model_type": "LSTM", \
        "vocab_size": INPUT_DIM, \
        "embedding_size": 40, \
        "hidden_size": 50, \
        "num_layers": 2, \
        "n_categories": 5, \
        "dropout": 0.5
    }
elif WHICH_TASK == "product":
    parameters = {
        "model_type": "LSTM", \
        "vocab_size": INPUT_DIM, \
        "embedding_size": 40, \
        "hidden_size": 50, \
        "num_layers": 2, \
        "n_categories": 18, \
        "dropout": 0.5
    }

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [ ]:
# it = iter(train_data)
# for i, b in enumerate(it):
#     print(b.narrative)
#     if i == 3:
#         break
# b0.label

In [90]:
# load models
if WHICH_TASK == "product":
    # full_product = util.load_model(parameters, "full_product_LSTM.pt")
    # sample_product = util.load_model(parameters, "trained_model_product.pt")
elif WHICH_TASK == "response"
    sample_response = util.load_model(parameters, "trained_model.pt")

In [97]:
util.compute_accuracy(sample_product, test_iter)